<a href="https://colab.research.google.com/github/jsroa15/BCG/blob/main/Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Churn prediction: Model and Evaluation**

# **Importing packages and modules**

In [1]:
#Necesary importings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Machine learning models

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB

#Machine learning utilities
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# **Loading data**


In [2]:
X_train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BCG/X_train.csv')
X_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BCG/X_test.csv')
y_train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BCG/y_train.csv')
y_test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BCG/y_test.csv')



In the previous EDA we discovered that original data is unbalanced. We have 90% of customers with retention and only 10% of customers with churn.

Based on that input, we are going to balance the data with synthetic samples.

In [3]:
#Fixing issue with unbalanced data. Generation of synthetic samples

#Methodology based on: https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18


from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=123, ratio=1)
X_train_smote, y_train_smote = sm.fit_sample(X_train, y_train)

#X_train_smote=pd.DataFrame(X_train_smote,columns=X_train.columns)
#y_train_smote=pd.DataFrame(y_train_smote,columns=y_train.columns)

X_train=X_train.values
y_train=y_train.values.ravel()
X_test=X_test.values
y_test=y_test.values.ravel()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

In [4]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [5]:
#Checking

from collections import Counter

print(Counter(y_train)) #Categories on initial y_train

print(Counter(y_train_smote)) #Categories in final y_train

Counter({0: 11311, 1: 1227})
Counter({0: 11311, 1: 11311})


# **Modeling**

## **Baseline model: Naive Bayes classifier with unbalanced data**

We are going to create a function to measure the performance of a classification model.





In [6]:
def baseline_model_performance_classification(model,Xtrain,Ytrain,Xtest,Ytest):

  '''
  Calculate metrics as Confusion matrix, AUC, Accuracy, Precision, Recall, F1 Score

  **Note**: Before running the function it's necessary to intantiate a classification model

  Parameters
  ------------------------------

  model: Classification model created prior to apply function.
  Xtrain: Your X train dataset
  Ytrain: Your Y train dataset
  Xtest: Your X test dataset
  Ytest: Your Y test dataset

  '''

  #Fit the model

  model.fit(Xtrain,Ytrain)

  #Make predictions

  y_pred=model.predict(Xtest)
  y_pred_proba=model.predict_proba(Xtest)[:,1]

  #Printing Metrics

  print('\nConfusion Matrix')
  print(confusion_matrix(Ytest, y_pred))


  print('\nScores')
  print('------------------------')
  print('AUC:',np.round(roc_auc_score(Ytest,y_pred_proba)*100,2),'%')
  print('Accuracy:',np.round(accuracy_score(Ytest,y_pred)*100,2),'%')
  print('Precision:',np.round(precision_score(Ytest,y_pred)*100,2),'%')
  print('Recall:',np.round(recall_score(Ytest,y_pred)*100,2),'%')
  print('F1 score:',np.round(f1_score(Ytest,y_pred)*100,2))



In [7]:
#Call the new function

model=GaussianNB()
baseline_model_performance_classification(model,X_train,y_train,X_test,y_test)



Confusion Matrix
[[ 754 2088]
 [  55  238]]

Scores
------------------------
AUC: 59.5 %
Accuracy: 31.64 %
Precision: 10.23 %
Recall: 81.23 %
F1 score: 18.17


In [8]:
#Measure performance with balanced data

model=GaussianNB()
baseline_model_performance_classification(model,X_train_smote,y_train_smote,X_test,y_test)



Confusion Matrix
[[ 925 1917]
 [  67  226]]

Scores
------------------------
AUC: 58.71 %
Accuracy: 36.71 %
Precision: 10.55 %
Recall: 77.13 %
F1 score: 18.56


From the above we can conclude that when there is balanced data to train the model, metrics like AUC and Accuracy decreased a little bit.

Now let's see how a robust model looks like.

## **Robust models**

Before moving forward, it's necessary to define the proper metric the evaluate our model. In our case we are going to take ROC AUC metric to compare and assess models.

The following is a function to perform that evaluation with cross validation and validation in test set.

In [9]:
def performance_classification_auc(model_dictionary,Xtrain,ytrain,Xtest,ytest,folds=5):
  '''
  Evaluate the performance of classification models via CV. The base metric is ROC AUC score

  Parameters
  ------------
    model_dictionary: A dictionary that contains as key the instantiate model and as value the model name
    Xtrain: The X train dataset
    ytrain: The y train dataset
    Xtest: The X test dataset
    ytest: The y test dataset
    folds: Number of folds to cross-validate. Default 3

  '''
  null_list1=[]
  null_list2=[]

  for key,value in model_dictionary.items():
    metric=cross_val_score(key,Xtrain,ytrain,cv=folds,scoring='roc_auc')
    
    #Store the F1 score from CV
    metric_mean=round(metric.mean()*100,2)


    #Fit and predict
    key.fit(Xtrain,ytrain)
    y_pred=key.predict_proba(Xtest)[:,1]

    #Calculate the ROC AUC Score in Test
    metric_test=round(roc_auc_score(y_test,y_pred)*100,2)
    null_list1.append(metric_mean)
    null_list2.append(metric_test)

  index=[x for x in model_dictionary.values()]
  dff=pd.DataFrame([null_list1,null_list2],index=['ROC AUC Cross Val','ROC AUC Test set']).transpose()
  dff.index=index
  print(dff)

In [10]:
xg=XGBClassifier(random_state=123)
rf=RandomForestClassifier()
dt=DecisionTreeClassifier(random_state=123)
gb=GradientBoostingClassifier(random_state=123)
ad=AdaBoostClassifier(random_state=123)

models={xg:'Xgboost',rf:'Random Forest',dt:'Decision Tree',gb:'GradientBoost',ad:'Adaboosting'}

performance_classification_auc(models,X_train,y_train,X_test,y_test,folds=10)

               ROC AUC Cross Val  ROC AUC Test set
Xgboost                    68.50             67.76
Random Forest              68.54             69.94
Decision Tree              56.74             58.75
GradientBoost              68.22             66.46
Adaboosting                65.47             65.58


In [11]:
%time performance_classification_auc(models,X_train_smote,y_train_smote,X_test,y_test,folds=10)

               ROC AUC Cross Val  ROC AUC Test set
Xgboost                    95.78             66.46
Random Forest              98.10             68.72
Decision Tree              89.49             57.16
GradientBoost              95.68             65.35
Adaboosting                94.82             61.81
CPU times: user 5min 1s, sys: 188 ms, total: 5min 1s
Wall time: 5min 2s


From the above,we can see that classifiers that were trained with unbalanced data present difficulties to classify the customer.

On the other hand, classifiers with balanced data have a good performance in CV but have a bad performance in Test set, which shows that giving more data and try to balance it it's not worthy.

The strategy to follow is to make hyper parameter tuning and then use emsemble to improve performance.

## **Hyperparameter tuning**

Based on the previous results, the best models are Xgboost and Random Forest.

Let's setup hyperparameters

### Random Forest

In [12]:
#Exploring Random Forest parameters

rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [13]:
#Some parameters to evaluate

rf_params={
  'max_depth':[3,6,9,12],
  'n_estimators':[100,200,500],
  'max_features':['log2','auto','sqrt'],
  'min_samples_leaf':[2,10,30,40],
  'random_state':[123]
}

rf_cv=RandomizedSearchCV(rf,rf_params,scoring='roc_auc',cv=3)

%time rf_cv.fit(X_train,y_train)


CPU times: user 1min 33s, sys: 102 ms, total: 1min 33s
Wall time: 1min 33s


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [14]:
#Extracting best score of ROC AUC

print('ROC AUC: ',round(rf_cv.best_score_*100,3))

ROC AUC:  68.972


From the above we can see that the improvement is very small, now we can try the same aproach with balanced dataset.

Due to computational time, this time we are gonna use the Radomized Grid Search

In [15]:
rf_params={
  'max_depth':[3,6,9,12],
  'n_estimators':[100,200,500],
  'max_features':['log2','auto','sqrt'],
  'min_samples_leaf':[2,10,30,40],
  'random_state':[123]
}

rf_cv_rand=RandomizedSearchCV(rf,rf_params,scoring='roc_auc',cv=3)

%time rf_cv_rand.fit(X_train_smote,y_train_smote)

CPU times: user 3min 5s, sys: 217 ms, total: 3min 5s
Wall time: 3min 6s


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [16]:
#Extracting best score of ROC AUC

print('ROC AUC: ',round(rf_cv_rand.best_score_*100,3))

ROC AUC:  97.108


### XGBoost Classifier

Now we are going to tune parameters for XGboost

In [17]:
#Extract parameters

xg.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 123,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [18]:
xg_params={
'min_child_weight': [i for i in np.arange(1,15,1)],
 'gamma': [i for i in np.arange(0,6,0.5)],
 'subsample': [i for i in np.linspace(0.3,1,15)],
 'colsample_bytree': [i for i in np.linspace(0.3,1,15)],
 'max_depth': [i for i in np.arange(1,15,2)],
 'scale_pos_weight':[i for i in np.arange(1,15,1)],
 'learning_rate': [i for i in np.arange(0,0.15,0.01)],
 'n_estimators' : [i for i in np.arange(0,2000,100)],
 'colsample_bylevel': [i for i in np.linspace(0.3,1,15)],
 'colsample_bynode': [i for i in np.linspace(0.3,1,15)],


}

xg_cv_rand=RandomizedSearchCV(xg,xg_params,scoring='roc_auc',cv=5)

xg_cv_rand.fit(X_train,y_train)

print('ROC AUC: ',round(xg_cv_rand.best_score_*100,3))



ROC AUC:  69.463


In [19]:
model_xg=xg_cv_rand.best_estimator_
model_rf=rf_cv.best_estimator_

## Emsembling methods

In order to increase the performance, we are going to use voting classifier.



In [20]:
# Define the list classifiers

classifiers = [('XGBoost Classifier', model_xg), ('Random Forest',model_rf)]

#Instantiate Voting Classifier

vc = VotingClassifier(estimators=classifiers)


vc.fit(X_train, y_train)
y_pred_vc = vc.predict(X_test)

roc_auc=roc_auc_score(y_test,y_pred_vc)

print(round(roc_auc*100,2))


50.51


Voting classifier didn't give a significance improvement. Our Final model will be the Xgboost Classifier. 

In [21]:
y_pred=model_xg.predict_proba(X_test)[:,1]

roc_auc=roc_auc_score(y_test,y_pred)

print(round(roc_auc*100,2))

70.22


# **Final evaluation and conclusions**

## Feature importance for XGBoost Classifier

In [22]:
#Extracting feature importances for XGBoost Classifier

fi=model_xg.feature_importances_

aux=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BCG/X_train.csv')

fi=pd.DataFrame(fi,columns=['feature_importance'],index=aux.columns)

fi.sort_values(by='feature_importance',ascending=False)



,feature_importance
origin_up_lxidpiddsbxsbosboudacockeimpuepw,0.081669
origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,0.078143
channel_sales_foosdfpfkusacimwkcsosbicdxkicaua,0.042566
margin_net_pow_ele,0.036188
margin_gross_pow_ele,0.032878
date_end_qtr_Q4,0.032743
total_cons,0.030357
channel_sales_lmkebamcaaclubfxadlmueccxoimlema,0.030285
date_end_qtr_Q3,0.029947
channel_sales_usilxuppasemubllopkaafesmlibmsdf,0.029790


In [23]:
#Model Evaluation

baseline_model_performance_classification(model_xg,X_train,y_train,X_test,y_test)


Confusion Matrix
[[2459  383]
 [ 167  126]]

Scores
------------------------
AUC: 70.22 %
Accuracy: 82.46 %
Precision: 24.75 %
Recall: 43.0 %
F1 score: 31.42


In [24]:
y_pred=model_xg.predict_proba(X_test)[:,1]
y_pred_pro=y_pred>0.65
print(roc_auc_score(y_test,y_pred))
print(precision_score(y_test,y_pred_pro))
print(recall_score(y_test,y_pred_pro))
print(f1_score(y_test,y_pred_pro))

0.7021769988447303
0.4369747899159664
0.17747440273037543
0.25242718446601947
